# Final analysis

This notebook contains the final form of our analysis as well as the final form of the data cleaning process.

In [2]:
from utils import *

In [51]:
path = r'/home/daniel/code/costofcare/data/bhp_ukhls_data_stata/UKDA-6614-stata/stata/ukhls'
my_colnames = [
                'hidp',
                'sex',
                'jbnssec_dv',
                'jbnssec8_dv',
                'jbstat',
                'jbhrs',
                'jbft_dv',
                'ethn_dv',
                'dvage',
                'mastat_dv',
                'country',
                'aidhh',
                'aidhrs',
                'aidxhh',
                'paygu_dv',
                'qfhigh',
                'qfhigh_dv',
                'istrtdatm',
                'istrtdaty',
                'month',
                'indinus_lw',
                'indinub_lw',
                'indinui_lw'
]

In [52]:
data = load_data(path, my_colnames)

In [53]:
cpih = pd.read_csv('./cpih_uk_88_23.csv')
data_clean = recoding_and_cleaning(data, cpih)

/home/daniel/code/costofcare/utils.py:329: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['jbhrs_clean'][data.jbhrs_clean < 1] = np.nan


In [54]:
data_clean['treated'] = data_clean['aidhrs_recoded_3'].transform(pd.notnull)
data_clean['control'] = data_clean['aidhrs'] == 'inapplicable'
data_clean['ever_treated'] = data_clean.groupby('pidp')['treated'].transform(any).values
data_clean.to_csv('data_clean.csv')

In [55]:
data_clean['edu_3'] = data_clean.edu_3.replace({1.0: 'low', 2.0:'middle', 3.0:'high'})

In [56]:
dummies = pd.get_dummies(data_clean.ethn_5)
edu_dumm = pd.get_dummies(data_clean.edu_3)
data_final = pd.concat([data_clean, dummies, edu_dumm], axis=1)

In [57]:
data_final

,pidp,hidp,sex,jbnssec_dv,jbnssec8_dv,jbstat,jbhrs,jbft_dv,ethn_dv,dvage,...,control,ever_treated,asian,black,mixed,other,white,high,low,middle
0,68001367,68001363,male,2 Higher managerial and admin. Occupations,Large employers & higher management,Paid employment(ft/pt),37.0,FT employee,british/english/scottish/welsh/northern irish,39.0,...,True,False,0,0,0,0,1,0,0,1
1,68004087,68004083,male,12.1 Semi-routine sales operations,Semi-routine,Paid employment(ft/pt),39.0,FT employee,any other white background,59.0,...,True,True,0,0,0,0,1,0,0,1
2,68006127,68006123,female,inapplicable,inapplicable,unemployed,inapplicable,inapplicable,british/english/scottish/welsh/northern irish,39.0,...,True,True,0,0,0,0,1,0,1,0
3,68006135,68006123,female,inapplicable,inapplicable,full-time student,inapplicable,inapplicable,british/english/scottish/welsh/northern irish,17.0,...,True,True,0,0,0,0,1,0,0,1
4,68006807,68006803,female,inapplicable,inapplicable,retired,inapplicable,inapplicable,british/english/scottish/welsh/northern irish,72.0,...,True,False,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505452,1653059690,1637331222,female,7.3 Intermediate techn. & auxiliary occupations,Intermediate,Paid employment(ft/pt),44.0,FT employee,indian,58.0,...,True,False,1,0,0,0,0,0,0,0
505453,1653134450,1637351622,female,inapplicable,inapplicable,Family care or home,inapplicable,inapplicable,indian,48.0,...,True,False,1,0,0,0,0,0,0,0
505454,1653134490,1637351622,male,3.4 Higher professional: 'new' self-employed,Higher professional,Self employed,inapplicable,FT employee,indian,51.0,...,True,False,1,0,0,0,0,0,0,0
505455,1653134530,1637351622,female,inapplicable,inapplicable,Full-time student,inapplicable,inapplicable,indian,21.0,...,True,False,1,0,0,0,0,1,0,0


In [58]:
conds = {
    'employed': True,
    'dropna': True,
    'target_var': 'log_wage_h_deflated',
    'covariates': [],
    'min_treat_waves': 2,
    'min_waves_pretreat': 4
}

In [59]:
treated, control = isc_data_preparation(data_final, conds)

In [60]:
treated.to_csv('treated.csv')
control.to_csv('control.csv')

In [27]:
data_final.jbnssec8_dv.value_counts()

inapplicable                           219244
Lower management & professional         79918
Semi-routine                            49982
Intermediate                            37753
Small employers & own account           28384
Routine                                 27229
Higher professional                     23775
Lower supervisory & technical           20023
Large employers & higher management     12523
missing                                  6626
Name: jbnssec8_dv, dtype: int64